We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [13]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))
print(data)
data.to_csv('email_modified.csv')

                                                   class  \
emails/spam\00001.7848dde101aa985090474a91ec93fcf0  spam   
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09  spam   
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c  spam   
emails/spam\00004.eac8de8d759b7e74154f142194282724  spam   
emails/spam\00005.57696a39d7d84318ce497886896bf90d  spam   
emails/spam\00006.5ab5620d3d7c6c0db76234556a16f6c1  spam   
emails/spam\00007.d8521faf753ff9ee989122f6816f87d7  spam   
emails/spam\00008.dfd941deb10f5eed78b1594b131c9266  spam   
emails/spam\00009.027bf6e0b0c4ab34db3ce0ea4bf2edab  spam   
emails/spam\00010.445affef4c70feec58f9198cfbc22997  spam   
emails/spam\00011.61816b9ad167657773a427d890d0468e  spam   
emails/spam\00012.381e4f512915109ba1e0853a7a8407b2  spam   
emails/spam\00013.d3f0b591a65f116ea5d9d4ad919f83aa  spam   
emails/spam\00014.7d38c46424f24fc8012ac15a95a2ac14  spam   
emails/spam\00015.048434ab64c86cf890eda1326a5643f5  spam   
emails/spam\00016.67fb281761ca1051a22ec3

Let's have a look at that DataFrame:

In [14]:
df = pd.read_csv('email_modified.csv')
print(df)

                                             Unnamed: 0 class  \
0     emails/spam\00001.7848dde101aa985090474a91ec93...  spam   
1     emails/spam\00002.d94f1b97e48ed3b553b3508d116e...  spam   
2     emails/spam\00003.2ee33bc6eacdb11f38d052c44819...  spam   
3     emails/spam\00004.eac8de8d759b7e74154f14219428...  spam   
4     emails/spam\00005.57696a39d7d84318ce497886896b...  spam   
5     emails/spam\00006.5ab5620d3d7c6c0db76234556a16...  spam   
6     emails/spam\00007.d8521faf753ff9ee989122f6816f...  spam   
7     emails/spam\00008.dfd941deb10f5eed78b1594b131c...  spam   
8     emails/spam\00009.027bf6e0b0c4ab34db3ce0ea4bf2...  spam   
9     emails/spam\00010.445affef4c70feec58f9198cfbc2...  spam   
10    emails/spam\00011.61816b9ad167657773a427d890d0...  spam   
11    emails/spam\00012.381e4f512915109ba1e0853a7a84...  spam   
12    emails/spam\00013.d3f0b591a65f116ea5d9d4ad919f...  spam   
13    emails/spam\00014.7d38c46424f24fc8012ac15a95a2...  spam   
14    emails/spam\00015.0

Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [15]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [16]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')